In [1]:
import warnings
warnings.filterwarnings('ignore')

import argparse
import time

import numpy as np
from scipy.sparse import coo_matrix, csc_matrix
from scipy.sparse.linalg import svds

import torch
from input_data import load_data
from preprocessing import general_train_test_split_edges,biased_train_test_split_edges, bidirectional_train_test_split_edges
from layers import DirectedGCNConvEncoder, DirectedInnerProductDecoder
from layers import SingleLayerDirectedGCNConvEncoder
from models import DirectedGAE

from layers import InnerProductDecoder
from layers import DummyEncoder, DummyPairEncoder

from models import GAE

In [2]:
parser = argparse.ArgumentParser()

parser.add_argument('--dataset',
                    nargs= '?',
                    default='cornell',
                    type=str)
parser.add_argument('--task',
                    nargs= '?',
                    default='task_2',
                    type=str)
parser.add_argument('--model',
                    nargs= '?',
                    default='digae',
                    type=str)
parser.add_argument('--learning_rate',
                    nargs= '?',
                    default=0.01,
                    type=float)
parser.add_argument('--epochs',
                    nargs= '?',
                    default=200,
                    type=int)
parser.add_argument('--hidden',
                    nargs= '?',
                    default=64,
                    type=int)
parser.add_argument('--dimension',
                    nargs= '?',
                    default=32,
                    type=int)
parser.add_argument('--alpha',
                    nargs= '?',
                    default=1.0,
                    type=float)
parser.add_argument('--beta',
                    nargs= '?',
                    default=0.0,
                    type=float)
parser.add_argument('--nb_run',
                    nargs= '?',
                    default=10,
                    type=int)
parser.add_argument('--prop_val',
                    nargs= '?',
                    default=5.0,
                    type=float)
parser.add_argument('--prop_test',
                    nargs= '?',
                    default=10.0,
                    type=float)
parser.add_argument('--verbose',
                    nargs= '?',
                    default=True,
                    type=bool)

parser.add_argument('--self_loops',
                    nargs= '?',
                    default=True,
                    type=bool)
parser.add_argument('--adaptive',
                    nargs= '?',
                    default=False,
                    type=bool)
parser.add_argument('--feature_vector_type',
                    nargs= '?',
                    const=None)
parser.add_argument('--feature_vector_size',
                    nargs= '?',
                    const=None,
                    type=int)
parser.add_argument('--directed',
                    nargs= '?',
                    default=True,
                    type=bool)
parser.add_argument('--logfile',
                    nargs='?',
                    default='logs.json',
                    type=str)
parser.add_argument('--validate',
                    nargs='?',
                    default=False,
                    type=bool)

args,_ = parser.parse_known_args()

In [3]:
def train_single():
    model.train()
    optimizer.zero_grad()
    z = model.encode(x, train_pos_edge_index)
    loss = model.recon_loss(z, train_pos_edge_index)
    loss.backward()
    optimizer.step()
    return float(loss)


def test_single(pos_edge_index, neg_edge_index):
    model.eval()
    with torch.no_grad():
        z = model.encode(x, train_pos_edge_index)
    return model.test(z, pos_edge_index, neg_edge_index)

def dummy_train_single():
    model.train()
    z = model.encode(x, train_pos_edge_index)
    loss = model.recon_loss(z, train_pos_edge_index)
    return float(loss)

def dummy_test_single(pos_edge_index, neg_edge_index):
    model.eval()
    with torch.no_grad():
        z = model.encode(x, train_pos_edge_index)
    return model.test(z, pos_edge_index, neg_edge_index)
    

def train_pair():
    model.train()
    optimizer.zero_grad()
    s, t = model.encode(u, v, train_pos_edge_index)
    loss = model.recon_loss(s, t, train_pos_edge_index)
    loss.backward()
    optimizer.step()
    return float(loss)

def test_pair(pos_edge_index, neg_edge_index):
    model.eval()
    with torch.no_grad():
        s, t = model.encode(u, v, train_pos_edge_index)
    return model.test(s, t, pos_edge_index, neg_edge_index)

def dummy_train_pair():
    model.train()
    s, t = model.encode(u, v, train_pos_edge_index)
    loss = model.recon_loss(s, t, train_pos_edge_index)
    return float(loss)

def dummy_test_pair(pos_edge_index, neg_edge_index):
    model.eval()
    with torch.no_grad():
        s, t = model.encode(u, v, train_pos_edge_index)
    return model.test(s, t, pos_edge_index, neg_edge_index)

def identity_features(data, k=None):
    num_nodes = data.x.size(0)
    x = torch.eye(num_nodes)
    u_hat = x.clone()
    v_hat = x.clone()
    return u_hat, v_hat

In [4]:
dummy_run = False
 
if args.verbose:
    print("Loading data...")

device     = 'cpu'
directed   = args.directed

dataset_name    = args.dataset
out_channels    = args.dimension
hidden_channels = args.hidden

compute_features = identity_features

feature_vector_size = args.feature_vector_size

if args.model in ['gae','source_target']:
    train_func = train_single
    test_func  = test_single
elif args.model in ['dummy']:
    train_func = dummy_train_single
    test_func  = dummy_test_single
elif args.model in ['dummy_pair']:
    train_func = dummy_train_pair
    test_func  = dummy_test_pair
else:
    train_func = train_pair
    test_func  = test_pair

val_ratio  = args.prop_val / 100.
test_ratio = args.prop_test / 100.

loaded_data = load_data(dataset_name, directed=directed)

Loading data...


In [5]:
mean_roc  = []
mean_ap   = []
mean_time = []

for i in range(args.nb_run):
    if args.verbose:
        print("Masking test edges...")

    if args.task == 'task_1':
        data = loaded_data.clone()
        data.train_mask = data.val_mask = data.test_mask = data.y = None
        data            = general_train_test_split_edges(data,
                                                         val_ratio=val_ratio,
                                                         test_ratio=test_ratio,
                                                         directed=directed)
    elif args.task == 'task_2':
        data = loaded_data.clone()
        data.train_mask = data.val_mask = data.test_mask = data.y = None
        data            = biased_train_test_split_edges(data,
                                                        val_ratio=val_ratio,
                                                        test_ratio=test_ratio,
                                                        directed=directed)
    elif args.task == 'task_3':
        data = loaded_data.clone()
        data.train_mask = data.val_mask = data.test_mask = data.y = None
        data            = bidirectional_train_test_split_edges(data,
                                                               val_ratio=val_ratio,
                                                               test_ratio=test_ratio,
                                                               directed=directed)     
    else:
        raise ValueError('Undefined task!')

    data = data.to(device)
    train_pos_edge_index = data.train_pos_edge_index.to(device)
    if args.validate is True:
        test_pos_edge_index  = data.val_pos_edge_index.to(device)
        test_neg_edge_index  = data.val_neg_edge_index.to(device)
    else:
        test_pos_edge_index  = data.test_pos_edge_index.to(device)
        test_neg_edge_index  = data.test_neg_edge_index.to(device)

    in_channels          = data.x.shape[1]

    in_channels = data.x.size(0)
    u, v = compute_features(data, in_channels)
    data.u = u
    data.v = v
    data.x = torch.cat([data.u, data.v], dim=1)
    
    u = data.u.to(device)
    v = data.v.to(device)
    print(u.shape, v.shape)

    x = data.x.to(device)

    if args.model == 'digae':
        encoder = DirectedGCNConvEncoder(in_channels, hidden_channels, out_channels, alpha=args.alpha, beta=args.beta,
                                         self_loops=args.self_loops,
                                         adaptive=args.adaptive)
        decoder = DirectedInnerProductDecoder()
        model   = DirectedGAE(encoder, decoder)
        model   = model.to(device)

    elif args.model == 'digae_single_layer':
        encoder = SingleLayerDirectedGCNConvEncoder(in_channels, out_channels, alpha=args.alpha, beta=args.beta,
                                                    self_loops=args.self_loops,
                                                    adaptive=args.adaptive)
        decoder = DirectedInnerProductDecoder()
        model   = DirectedGAE(encoder, decoder)
        model   = model.to(device)

    elif args.model == 'dummy':
        encoder = DummyEncoder()
        decoder = InnerProductDecoder()
        model   = GAE(encoder, decoder)
        model   = model.to(device)
        dummy_run = True

    elif args.model == 'dummy_pair':
        encoder = DummyPairEncoder()
        decoder = DirectedInnerProductDecoder()
        model   = DirectedGAE(encoder, decoder)
        model   = model.to(device)
        dummy_run =True
    else:
        raise ValueError('Undefined model!')

    optimizer = torch.optim.Adam(model.parameters(), lr=args.learning_rate, weight_decay=5e-4)

    if args.verbose:
        print("Training...")

    # Flag to compute total running time
    t_start = time.time()
    for epoch in range(args.epochs):
        # Flag to compute running time for each epoch
        t = time.time()
        loss = train_func()
        avg_cost = loss
        print("Epoch:", '%04d' % (epoch + 1), "train_loss=", "{:.5f}".format(avg_cost),
        "time=", "{:.5f}".format(time.time() - t))
    mean_time.append(time.time() - t_start)

    if args.verbose:
        print("Testing model...")
    auc, ap = test_func(test_pos_edge_index, test_neg_edge_index)
    roc_score, ap_score = auc, ap
    mean_roc.append(roc_score)
    mean_ap.append(ap_score)

# if adaptive...
print('=' * 60)
print(args.adaptive)
print(args.model)
print('=' * 60)
    
# Report final results
print("\nTest results for", args.model,
      "model on", args.dataset, "on", args.task, "\n",
      "___________________________________________________\n")

print("AUC scores\n", mean_roc)
print("Mean AUC score: ", np.mean(mean_roc),
      "\nStd of AUC scores: ", np.std(mean_roc), "\n \n")

print("AP scores \n", mean_ap)
print("Mean AP score: ", np.mean(mean_ap),
      "\nStd of AP scores: ", np.std(mean_ap), "\n \n")

print("Running times\n", mean_time)
print("Mean running time: ", np.mean(mean_time),
      "\nStd of running time: ", np.std(mean_time), "\n \n")

Masking test edges...
torch.Size([183, 183]) torch.Size([183, 183])
Training...
Epoch: 0001 train_loss= 1.38664 time= 0.01008
Epoch: 0002 train_loss= 1.38498 time= 0.00499
Epoch: 0003 train_loss= 1.38253 time= 0.00701
Epoch: 0004 train_loss= 1.37934 time= 0.00598
Epoch: 0005 train_loss= 1.37279 time= 0.00501
Epoch: 0006 train_loss= 1.35818 time= 0.00601
Epoch: 0007 train_loss= 1.34164 time= 0.00599
Epoch: 0008 train_loss= 1.32408 time= 0.00501
Epoch: 0009 train_loss= 1.30445 time= 0.00300
Epoch: 0010 train_loss= 1.26098 time= 0.00456
Epoch: 0011 train_loss= 1.27064 time= 0.00413
Epoch: 0012 train_loss= 1.23095 time= 0.00445
Epoch: 0013 train_loss= 1.22749 time= 0.00465
Epoch: 0014 train_loss= 1.20279 time= 0.00594
Epoch: 0015 train_loss= 1.21843 time= 0.00742
Epoch: 0016 train_loss= 1.19119 time= 0.00464
Epoch: 0017 train_loss= 1.17999 time= 0.00916
Epoch: 0018 train_loss= 1.14115 time= 0.00405
Epoch: 0019 train_loss= 1.10919 time= 0.00347
Epoch: 0020 train_loss= 1.13902 time= 0.00200


Epoch: 0198 train_loss= 0.38618 time= 0.00800
Epoch: 0199 train_loss= 0.34260 time= 0.00470
Epoch: 0200 train_loss= 0.38070 time= 0.00800
Testing model...
Masking test edges...
torch.Size([183, 183]) torch.Size([183, 183])
Training...
Epoch: 0001 train_loss= 1.38645 time= 0.00697
Epoch: 0002 train_loss= 1.38472 time= 0.00621
Epoch: 0003 train_loss= 1.38106 time= 0.00595
Epoch: 0004 train_loss= 1.37709 time= 0.00421
Epoch: 0005 train_loss= 1.37066 time= 0.00572
Epoch: 0006 train_loss= 1.35735 time= 0.00594
Epoch: 0007 train_loss= 1.33914 time= 0.00606
Epoch: 0008 train_loss= 1.31883 time= 0.00500
Epoch: 0009 train_loss= 1.28837 time= 0.00698
Epoch: 0010 train_loss= 1.24719 time= 0.00702
Epoch: 0011 train_loss= 1.23600 time= 0.00601
Epoch: 0012 train_loss= 1.20266 time= 0.00692
Epoch: 0013 train_loss= 1.22053 time= 0.00548
Epoch: 0014 train_loss= 1.19638 time= 0.00491
Epoch: 0015 train_loss= 1.21678 time= 0.00700
Epoch: 0016 train_loss= 1.14001 time= 0.00500
Epoch: 0017 train_loss= 1.173

Epoch: 0182 train_loss= 0.37021 time= 0.00700
Epoch: 0183 train_loss= 0.33528 time= 0.00600
Epoch: 0184 train_loss= 0.43107 time= 0.00635
Epoch: 0185 train_loss= 0.31663 time= 0.00561
Epoch: 0186 train_loss= 0.27475 time= 0.00582
Epoch: 0187 train_loss= 0.28105 time= 0.00618
Epoch: 0188 train_loss= 0.36366 time= 0.00587
Epoch: 0189 train_loss= 0.35136 time= 0.00594
Epoch: 0190 train_loss= 0.31613 time= 0.00600
Epoch: 0191 train_loss= 0.42734 time= 0.00601
Epoch: 0192 train_loss= 0.32713 time= 0.00599
Epoch: 0193 train_loss= 0.32622 time= 0.00500
Epoch: 0194 train_loss= 0.37179 time= 0.00701
Epoch: 0195 train_loss= 0.33895 time= 0.00499
Epoch: 0196 train_loss= 0.32820 time= 0.00600
Epoch: 0197 train_loss= 0.44833 time= 0.00600
Epoch: 0198 train_loss= 0.31815 time= 0.00600
Epoch: 0199 train_loss= 0.36824 time= 0.00647
Epoch: 0200 train_loss= 0.33052 time= 0.00453
Testing model...
Masking test edges...
torch.Size([183, 183]) torch.Size([183, 183])
Training...
Epoch: 0001 train_loss= 1.387

Epoch: 0168 train_loss= 0.38244 time= 0.00691
Epoch: 0169 train_loss= 0.34232 time= 0.00601
Epoch: 0170 train_loss= 0.36263 time= 0.00601
Epoch: 0171 train_loss= 0.33715 time= 0.00699
Epoch: 0172 train_loss= 0.31300 time= 0.00499
Epoch: 0173 train_loss= 0.34655 time= 0.00603
Epoch: 0174 train_loss= 0.33965 time= 0.00598
Epoch: 0175 train_loss= 0.38056 time= 0.00600
Epoch: 0176 train_loss= 0.34176 time= 0.00599
Epoch: 0177 train_loss= 0.32210 time= 0.00600
Epoch: 0178 train_loss= 0.36049 time= 0.00600
Epoch: 0179 train_loss= 0.30571 time= 0.00546
Epoch: 0180 train_loss= 0.24959 time= 0.00455
Epoch: 0181 train_loss= 0.42590 time= 0.00342
Epoch: 0182 train_loss= 0.33488 time= 0.00694
Epoch: 0183 train_loss= 0.31345 time= 0.00804
Epoch: 0184 train_loss= 0.38405 time= 0.00694
Epoch: 0185 train_loss= 0.36235 time= 0.00600
Epoch: 0186 train_loss= 0.34538 time= 0.00800
Epoch: 0187 train_loss= 0.32637 time= 0.00700
Epoch: 0188 train_loss= 0.32371 time= 0.00700
Epoch: 0189 train_loss= 0.37490 ti

Epoch: 0154 train_loss= 0.35284 time= 0.00622
Epoch: 0155 train_loss= 0.31289 time= 0.00800
Epoch: 0156 train_loss= 0.35169 time= 0.00800
Epoch: 0157 train_loss= 0.31779 time= 0.00600
Epoch: 0158 train_loss= 0.36885 time= 0.00700
Epoch: 0159 train_loss= 0.33848 time= 0.00600
Epoch: 0160 train_loss= 0.31659 time= 0.00700
Epoch: 0161 train_loss= 0.31787 time= 0.00601
Epoch: 0162 train_loss= 0.37557 time= 0.00498
Epoch: 0163 train_loss= 0.38173 time= 0.00619
Epoch: 0164 train_loss= 0.34954 time= 0.00598
Epoch: 0165 train_loss= 0.42885 time= 0.00695
Epoch: 0166 train_loss= 0.33866 time= 0.00701
Epoch: 0167 train_loss= 0.33152 time= 0.00606
Epoch: 0168 train_loss= 0.44202 time= 0.00794
Epoch: 0169 train_loss= 0.31453 time= 0.00613
Epoch: 0170 train_loss= 0.33522 time= 0.00700
Epoch: 0171 train_loss= 0.33989 time= 0.00600
Epoch: 0172 train_loss= 0.36235 time= 0.00600
Epoch: 0173 train_loss= 0.40192 time= 0.00715
Epoch: 0174 train_loss= 0.28373 time= 0.00407
Epoch: 0175 train_loss= 0.40752 ti

...

Epoch: 0136 train_loss= 0.37655 time= 0.00592
Epoch: 0137 train_loss= 0.36887 time= 0.00469
Epoch: 0138 train_loss= 0.36552 time= 0.00695
Epoch: 0139 train_loss= 0.32038 time= 0.00599
Epoch: 0140 train_loss= 0.35484 time= 0.00518
Epoch: 0141 train_loss= 0.41345 time= 0.00706
Epoch: 0142 train_loss= 0.34326 time= 0.00701
Epoch: 0143 train_loss= 0.34516 time= 0.00693
Epoch: 0144 train_loss= 0.33878 time= 0.00500
Epoch: 0145 train_loss= 0.33367 time= 0.00700
Epoch: 0146 train_loss= 0.23714 time= 0.00595
Epoch: 0147 train_loss= 0.39815 time= 0.00699
Epoch: 0148 train_loss= 0.30654 time= 0.00709
Epoch: 0149 train_loss= 0.41677 time= 0.00790
Epoch: 0150 train_loss= 0.35210 time= 0.00600
Epoch: 0151 train_loss= 0.40529 time= 0.00600
Epoch: 0152 train_loss= 0.35120 time= 0.00500
Epoch: 0153 train_loss= 0.34264 time= 0.00601
Epoch: 0154 train_loss= 0.35992 time= 0.00569
Epoch: 0155 train_loss= 0.35057 time= 0.00610
Epoch: 0156 train_loss= 0.37001 time= 0.00596
Epoch: 0157 train_loss= 0.45558 ti

Epoch: 0131 train_loss= 0.46531 time= 0.00682
Epoch: 0132 train_loss= 0.46138 time= 0.00140
Epoch: 0133 train_loss= 0.48461 time= 0.01086
Epoch: 0134 train_loss= 0.46027 time= 0.00670
Epoch: 0135 train_loss= 0.42264 time= 0.00617
Epoch: 0136 train_loss= 0.43572 time= 0.00611
Epoch: 0137 train_loss= 0.37675 time= 0.00500
Epoch: 0138 train_loss= 0.38701 time= 0.00594
Epoch: 0139 train_loss= 0.41156 time= 0.00606
Epoch: 0140 train_loss= 0.36260 time= 0.00500
Epoch: 0141 train_loss= 0.43555 time= 0.00657
Epoch: 0142 train_loss= 0.35385 time= 0.00107
Epoch: 0143 train_loss= 0.39948 time= 0.00106
Epoch: 0144 train_loss= 0.45959 time= 0.00791
Epoch: 0145 train_loss= 0.38195 time= 0.00354
Epoch: 0146 train_loss= 0.39497 time= 0.00582
Epoch: 0147 train_loss= 0.42126 time= 0.00867
Epoch: 0148 train_loss= 0.33983 time= 0.00401
Epoch: 0149 train_loss= 0.43168 time= 0.00538
Epoch: 0150 train_loss= 0.37239 time= 0.00647
Epoch: 0151 train_loss= 0.38958 time= 0.00107
Epoch: 0152 train_loss= 0.34943 ti

Epoch: 0126 train_loss= 0.40846 time= 0.00690
Epoch: 0127 train_loss= 0.41138 time= 0.00600
Epoch: 0128 train_loss= 0.40578 time= 0.00700
Epoch: 0129 train_loss= 0.46584 time= 0.00600
Epoch: 0130 train_loss= 0.40694 time= 0.00380
Epoch: 0131 train_loss= 0.42474 time= 0.00611
Epoch: 0132 train_loss= 0.43727 time= 0.00600
Epoch: 0133 train_loss= 0.44640 time= 0.00511
Epoch: 0134 train_loss= 0.43769 time= 0.00490
Epoch: 0135 train_loss= 0.38740 time= 0.00370
Epoch: 0136 train_loss= 0.41068 time= 0.00419
Epoch: 0137 train_loss= 0.38694 time= 0.00438
Epoch: 0138 train_loss= 0.41291 time= 0.00500
Epoch: 0139 train_loss= 0.36403 time= 0.00592
Epoch: 0140 train_loss= 0.34994 time= 0.00600
Epoch: 0141 train_loss= 0.35937 time= 0.00700
Epoch: 0142 train_loss= 0.32317 time= 0.00500
Epoch: 0143 train_loss= 0.50653 time= 0.00600
Epoch: 0144 train_loss= 0.41601 time= 0.00600
Epoch: 0145 train_loss= 0.41864 time= 0.00600
Epoch: 0146 train_loss= 0.40141 time= 0.00600
Epoch: 0147 train_loss= 0.47356 ti

Epoch: 0120 train_loss= 0.42769 time= 0.00200
Epoch: 0121 train_loss= 0.43930 time= 0.00306
Epoch: 0122 train_loss= 0.44661 time= 0.00538
Epoch: 0123 train_loss= 0.37574 time= 0.00513
Epoch: 0124 train_loss= 0.50035 time= 0.01115
Epoch: 0125 train_loss= 0.40113 time= 0.00500
Epoch: 0126 train_loss= 0.46815 time= 0.00472
Epoch: 0127 train_loss= 0.46154 time= 0.00622
Epoch: 0128 train_loss= 0.51137 time= 0.00495
Epoch: 0129 train_loss= 0.48386 time= 0.00643
Epoch: 0130 train_loss= 0.45018 time= 0.00557
Epoch: 0131 train_loss= 0.52358 time= 0.00500
Epoch: 0132 train_loss= 0.40287 time= 0.00600
Epoch: 0133 train_loss= 0.42902 time= 0.00601
Epoch: 0134 train_loss= 0.43198 time= 0.00499
Epoch: 0135 train_loss= 0.50878 time= 0.00600
Epoch: 0136 train_loss= 0.43309 time= 0.00500
Epoch: 0137 train_loss= 0.43417 time= 0.00390
Epoch: 0138 train_loss= 0.40243 time= 0.00533
Epoch: 0139 train_loss= 0.39279 time= 0.00500
Epoch: 0140 train_loss= 0.45514 time= 0.00601
Epoch: 0141 train_loss= 0.37647 ti

Epoch: 0110 train_loss= 0.43028 time= 0.00593
Epoch: 0111 train_loss= 0.41258 time= 0.00481
Epoch: 0112 train_loss= 0.38024 time= 0.00698
Epoch: 0113 train_loss= 0.39443 time= 0.00602
Epoch: 0114 train_loss= 0.44881 time= 0.00594
Epoch: 0115 train_loss= 0.39477 time= 0.00505
Epoch: 0116 train_loss= 0.44930 time= 0.00600
Epoch: 0117 train_loss= 0.38799 time= 0.00600
Epoch: 0118 train_loss= 0.39152 time= 0.00507
Epoch: 0119 train_loss= 0.43754 time= 0.00612
Epoch: 0120 train_loss= 0.43284 time= 0.00588
Epoch: 0121 train_loss= 0.31354 time= 0.00392
Epoch: 0122 train_loss= 0.42341 time= 0.00507
Epoch: 0123 train_loss= 0.34685 time= 0.00600
Epoch: 0124 train_loss= 0.35394 time= 0.00602
Epoch: 0125 train_loss= 0.40769 time= 0.00546
Epoch: 0126 train_loss= 0.40673 time= 0.00604
Epoch: 0127 train_loss= 0.40706 time= 0.00495
Epoch: 0128 train_loss= 0.36356 time= 0.00400
Epoch: 0129 train_loss= 0.39528 time= 0.00585
Epoch: 0130 train_loss= 0.40109 time= 0.00549
Epoch: 0131 train_loss= 0.35289 ti